# Car ride-share potential in mid-size U.S. cities from geographic spread

## (notebook 3: CITY_INDEX=6)

Third notebook for the IBM Data Science Specialization on Coursera, which contains the consolidated methods and functions from the first two notebooks, to serve as a template to be used per-city.



## Installs (if needed) and imports

See notesbooks 1 and 2 for details.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

try:
    from geopy.geocoders import Nominatim
    print(Nominatim)
except:
    !conda install -c conda-forge geopy=1.18.1 --yes
    
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import folium
    print(folium)
except:
    !conda install -c conda-forge folium=0.8.0 --yes
    
import folium
import time
    
try:
    import pandas_profiling
    print(pandas_profiling)
except:
    !conda install -c conda-forge pandas-profiling=1.4.1 --yes
    
import pandas_profiling

<class 'geopy.geocoders.osm.Nominatim'>
<module 'folium' from '/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/folium/__init__.py'>
<module 'pandas_profiling' from '/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas_profiling/__init__.py'>


## Foursquare secret

In [2]:
# (paste secret from local file)



In [3]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version

CLIENT_ID set: True
CLIENT_SECRET set: True


## Which city?

In [4]:
CITY_INDEX=6
RADIUS=800
MAX_COORDS=1000
MAP_ZOOM_LEVEL=11
MARKER_ZOOM=0.1

## All function definitions

See notebooks 1 and 2 for details.

In [5]:
print('Starting: Get list of cities')

url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })

print('---------CITIES TABLE (raw)')
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']

print('----------CITIES DATAFRAME')
print(cities_df)


def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False,
                       failsafe_abort_num=4000,
                       simple_rendering=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )
    
    num_markers=0

    # add markers to map
    for coords, venues in venues_dict.items():
        num_markers += 1
        if (venues.shape[0] > 0) and (num_markers < failsafe_abort_num): 
            if simple_rendering:
                folium.Circle(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    color=None,
                    fill=True,
                    fill_color='#005090',
                    fill_opacity=0.7).add_to(new_map)
            else:
                label = '{}, # venues: {}'.format(coords, venues.shape[0])
                label = folium.Popup(label, parse_html=True)
                folium.CircleMarker(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    popup=label,
                    color='blue',
                    fill=True,
                    fill_color='#3186cc',
                    fill_opacity=0.7,
                    parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map


def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


def gather_distance_distribution_stats(venues_agg_df, city_index=-1, city_display='(city name, state)', return_as_list_append_string=False):
    
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    num_venues_per_coord = venues_agg_df['num_venues']
    dist_per_coord = venues_agg_df['dist']
    
    # blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
    dist_list = []
    for this_coord in zip(num_venues_per_coord, dist_per_coord):
        this_list = [float(this_coord[1])] * int(this_coord[0])
        dist_list.extend(this_list)
    
    dist_list_df = pd.DataFrame(dist_list)
    dist = dist_list_df[0]
    
    # now perform statistics on that new list
    
    mean = dist.mean()
    std = dist.std()
    p25 = dist.quantile(0.25)
    median = dist.quantile(0.5)
    p75 = dist.quantile(0.75)
    max_d = dist.max()
    iqr = p75 - p25
    kurt = dist.kurtosis() # "peaky-ness"
    mad = dist.mad() # mean absolute deviation
    skew = dist.skew() # skewedness
    
    if return_as_list_append_string:
        retS = '# Create list at the beginning:\n'
        retS += '#     l = []\n\n'
        retS += 'l.append(['
        retS += '{}, "{}", ({:.2f}, {:.2f}), '.format(city_index, city_display, center_x, center_y)
        retS += '{}, {}, {}, {}, '.format(total_num_venues, total_num_indicators, mean, std)
        retS += '{}, {}, {}, {}, '.format(p25, median, p75, max_d)
        retS += '{}, {}, {}, {}'.format(iqr, kurt, mad, skew)
        retS += '])\n\n'
        retS += '# Convert list to pandas DataFrame:\n'
        retS += '#     cities_stats_df = pd.DataFrame(l)\n'
        retS += '#     cities_stats_df.columns = ['
        retS += '"city index", "city name, state", "center coord", "total num venues", "total num indicators", '
        retS += '"mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", '
        retS += '"interquartile range", "kurtosis", "mean absolute deviation", "skewedness"'
        retS += ']\n'
        return retS

    else:
        return center_x, center_y, total_num_venues, total_num_indicators, \
               mean, std, p25, median, p75, max_d,                         \
               iqr, kurt, mad, skew
    

    



Starting: Get list of cities
---------CITIES TABLE (raw)
<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></s

## Quick test

In [6]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=10, verbose=False)

print("{}, {}\n".format(city_name, city_state))

venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)

print("\n{}\n".format(venues_agg_df.head(10)))

center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew \
    = gather_distance_distribution_stats(venues_agg_df)

print(center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew)

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nlist_append_string:\n\n{}'.format(list_append_string))

make_map_from_dict(
    lat_orig, long_orig, venues_dict, 13, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)


[test #1 of 10] (0,0):0. [test #2 of 10] (-1,0):1. (1,0):0. (0,-1):0. (0,1):0. (-1,1):1. (1,-1):0. [test #3 of 10] (-2,0):(err)5. (-1,-1):2. (-2,1):2. [test #4 of 10] (-3,0):5. (-2,-1):5. (-3,1):5. [test #5 of 10] (-4,0):4. (-3,-1):4. (-4,1):5. [test #6 of 10] (-2,-2):3. (-1,-2):0. [test #7 of 10] (-3,2):4. (-4,2):5. [test #8 of 10] (-5,1):2. (-5,2):5. [test #9 of 10] (-4,3):2. (-5,3):2. [test #10 of 10] (-6,2):1. (-6,3):1. Aurora, Colorado


      coord  num_venues  num_indicators
0    (0, 0)           0               0
1   (-1, 0)           1               0
2  (-3, -1)           4               0
3  (-1, -1)           2               0
4  (-2, -1)           5               0
5   (-6, 3)           1               0
6   (-2, 1)           2               0
7   (-4, 2)           5               0
8   (-4, 3)           2               0
9   (-4, 0)           4               0

-3.265625 0.671875 64 0 1.6358326371259861 0.7901481811204386 0.9953505192895616 1.51763719519851 2.184484528727

## The real deal

In [7]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=MAX_COORDS, verbose=False)

print("\n\n{}, {}\n".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
print('Total shape: {}\n\nSample 10 venues per coordinate (unsorted):'.format(venues_agg_df.shape))
print('{}\n'.format(venues_agg_df.head(10)))

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nResulting list append string for this city:\n\n{}'.format(list_append_string))


[test #1 of 1000] (0,0):0. [test #2 of 1000] (-1,0):1. (1,0):0. (0,-1):0. (0,1):0. (-1,1):1. (1,-1):0. [test #3 of 1000] (-2,0):5. (-1,-1):2. (-2,1):2. [test #4 of 1000] (-3,0):5. (-2,-1):5. (-3,1):5. [test #5 of 1000] (-4,0):4. (-3,-1):4. (-4,1):5. [test #6 of 1000] (-2,-2):3. (-1,-2):0. [test #7 of 1000] (-3,2):4. (-4,2):5. [test #8 of 1000] (-5,1):2. (-5,2):5. [test #9 of 1000] (-4,3):2. (-5,3):2. [test #10 of 1000] (-6,2):1. (-6,3):1. [test #11 of 1000] (-5,0):3. (-4,-1):3. [test #12 of 1000] (-3,-2):2. [test #13 of 1000] (-2,2):1. (-3,3):0. [test #14 of 1000] (-2,-3):0. (-1,-3):0. [test #15 of 1000] (-6,0):0. (-5,-1):3. (-6,1):1. [test #16 of 1000] (-4,-2):3. [test #17 of 1000] (-6,-1):2. (-5,-2):3. [test #18 of 1000] (-4,-3):3. (-3,-3):1. [test #19 of 1000] (-6,-2):1. (-5,-3):2. [test #20 of 1000] (-4,-4):1. (-3,-4):1. [test #21 of 1000] (0,-2):0. [test #22 of 1000] [test #23 of 1000] [test #24 of 1000] (-4,4):2. (-5,4):4. [test #25 of 1000] (-6,4):1. (-5,5):3. (-6,5):3. [test #2

34. (-35,4):26. (-36,4):33. [test #203 of 1000] (-36,2):18. (-35,1):13. [test #204 of 1000] (-37,3):21. (-37,4):29. [test #205 of 1000] (-36,5):7. (-37,5):24. [test #206 of 1000] (-34,4):5. [test #207 of 1000] (-38,4):40. (-38,5):36. [test #208 of 1000] (-39,4):58. (-38,3):14. (-39,5):42. [test #209 of 1000] (-40,4):83. (-39,3):77. (-40,5):63. [test #210 of 1000] (-41,4):58. (-40,3):68. (-41,5):55. [test #211 of 1000] (-39,2):63. (-38,2):33. [test #212 of 1000] (-41,3):34. (-40,2):61. [test #213 of 1000] (-40,6):34. (-41,6):40. [test #214 of 1000] (-39,1):71. (-38,1):68. [test #215 of 1000] (-40,1):35. (-39,0):53. (-38,0):55. [test #216 of 1000] (-37,1):13. (-37,0):27. [test #217 of 1000] (-41,2):2. [test #218 of 1000] (-42,4):11. (-42,5):31. [test #219 of 1000] (-42,6):81. [test #220 of 1000] (-43,6):78. (-42,7):26. (-43,7):84. [test #221 of 1000] (-44,7):100. (-43,8):53. (-44,8):100. [test #222 of 1000] (-45,7):71. (-44,6):74. (-45,8):97. [test #223 of 1000] (-44,9):44. (-45,9):50. [

7. (-41,8):12. [test #406 of 1000] (-50,4):14. (-49,3):14. (-48,3):24. [test #407 of 1000] (-41,1):8. [test #408 of 1000] (-41,-1):52. (-40,-2):49. (-41,0):16. [test #409 of 1000] (-42,-1):43. (-41,-2):59. (-42,0):20. [test #410 of 1000] (-42,-2):40. (-41,-3):46. (-40,-3):50. [test #411 of 1000] (-40,-4):13. (-39,-4):12. [test #412 of 1000] [test #413 of 1000] (-42,-3):38. (-41,-4):10. [test #414 of 1000] (-43,-1):35. (-43,0):14. [test #415 of 1000] (-43,-2):35. [test #416 of 1000] (-43,-3):41. (-42,-4):13. [test #417 of 1000] (-44,-3):(err)39. (-43,-4):26. (-44,-2):(err)45. [test #418 of 1000] (-45,-2):45. (-44,-1):51. (-45,-1):51. [test #419 of 1000] (-44,0):41. (-45,0):86. [test #420 of 1000] (-46,0):45. (-45,1):68. (-46,1):76. [test #421 of 1000] (-47,1):42. (-46,2):54. (-47,2):39. [test #422 of 1000] (-44,1):47. (-45,2):34. [test #423 of 1000] [test #424 of 1000] (-46,-1):42. [test #425 of 1000] (-43,1):27. (-44,2):37. [test #426 of 1000] (-46,-2):16. (-45,-3):34. [test #427 of 10

11. [test #624 of 1000] [test #625 of 1000] (-59,22):18. [test #626 of 1000] [test #627 of 1000] (-46,-6):7. [test #628 of 1000] (-47,-8):17. (-47,-7):14. [test #629 of 1000] [test #630 of 1000] (-68,18):31. (-67,17):8. (-68,19):29. [test #631 of 1000] (-69,18):35. (-68,17):7. (-69,19):42. [test #632 of 1000] (-70,19):39. (-69,20):40. (-70,20):36. [test #633 of 1000] (-70,21):19. [test #634 of 1000] (-71,19):7. (-70,18):8. (-71,20):24. [test #635 of 1000] (-71,21):16. [test #636 of 1000] (-69,17):6. [test #637 of 1000] [test #638 of 1000] (-72,20):10. (-72,21):9. [test #639 of 1000] (-36,13):15. (-37,14):15. (-36,12):13. [test #640 of 1000] [test #641 of 1000] (-72,9):21. (-71,9):14. [test #642 of 1000] [test #643 of 1000] (-35,-2):16. [test #644 of 1000] [test #645 of 1000] [test #646 of 1000] (-44,15):16. (-45,16):17. [test #647 of 1000] [test #648 of 1000] (-47,-3):17. (-46,-4):16. (-47,-2):15. [test #649 of 1000] (-58,22):17. [test #650 of 1000] [test #651 of 1000] (-36,11):12. (-3

43. (-30,-7):44. (-31,-7):27. [test #834 of 1000] (-29,-7):41. (-30,-6):10. (-31,-6):19. [test #835 of 1000] (-28,-9):33. (-28,-10):7. [test #836 of 1000] (-28,-8):38. [test #837 of 1000] (-28,-7):17. (-29,-6):11. [test #838 of 1000] (-27,-8):26. (-27,-9):15. [test #839 of 1000] (-27,-10):14. [test #840 of 1000] (-32,-6):18. [test #841 of 1000] (-26,-8):20. (-27,-7):12. (-26,-9):15. [test #842 of 1000] [test #843 of 1000] [test #844 of 1000] (-25,-8):29. (-26,-7):16. (-25,-9):26. [test #845 of 1000] (-24,-8):24. (-25,-7):19. (-24,-9):32. [test #846 of 1000] (-23,-9):21. (-24,-10):9. (-23,-10):9. [test #847 of 1000] (-25,-10):7. [test #848 of 1000] [test #849 of 1000] (-22,-9):5. (-22,-10):4. [test #850 of 1000] (-31,-5):19. (-32,-5):17. [test #851 of 1000] (-26,-6):2. [test #852 of 1000] (-30,-5):5. (-31,-4):15. (-32,-4):27. [test #853 of 1000] (-33,-4):24. (-32,-3):42. (-33,-3):40. [test #854 of 1000] (-31,-3):24. (-32,-2):42. (-33,-2):28. [test #855 of 1000] (-31,-2):22. (-32,-1):10.

In [8]:
make_map_from_dict(
    lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)
